## убрать строки с нулевыми значениями Sales

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y.iloc[i] != 0:
            summ += (1-y_pred[i]/y.iloc[i])**2
    return sqrt(summ/len(y))

def substract_cols(df):
    df['CompetitionOpen'] = 12 * (df.Year - df['CompetitionOpenSinceYear']) + (df.Month - df['CompetitionOpenSinceMonth'])
    df['CompetitionOpen'] = df.CompetitionOpen.apply(lambda x: x if x > 0 else 0)
    df.loc[df['CompetitionOpenSinceYear'] < 0, 'CompetitionOpen'] = 0
    df['PromoOpen'] = 12 * (df.Year - df.Promo2SinceYear) + (df['WeekOfYear'] - df['Promo2SinceWeek']) / 4.0
    df['PromoOpen'] = df.PromoOpen.apply(lambda x: x if x > 0 else 0)
    df.loc[df['Promo2SinceYear'] == 0, 'PromoOpen'] = 0
    df.loc[df['Promo2SinceYear'] < 0, 'PromoOpen'] = 0
    df.drop(['CompetitionOpenSinceYear',
             'CompetitionOpenSinceMonth',
             'Promo2SinceYear',
             'Promo2SinceWeek'], axis=1, inplace=True)
    return df

In [10]:
df_train = pd.read_csv('data/train_v2.csv')
df_train['Date']  = pd.to_datetime(df_train['Date'], errors='coerce')
df_train['Year'] = df_train['Date'].dt.year
df_train['Month'] = df_train['Date'].dt.month
df_train['WeekOfYear'] = df_train['Date'].dt.weekofyear
df_train['Day'] = df_train['Date'].dt.day
df_train = df_train.drop('Date', axis=1)

df_test = pd.read_csv('data/test_v2.csv')
df_test['Date']  = pd.to_datetime(df_test['Date'], errors='coerce')
df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['WeekOfYear'] = df_test['Date'].dt.weekofyear
df_test['Day'] = df_test['Date'].dt.day
df_test = df_test.drop('Date', axis=1)

df_store = pd.read_csv('data/store.csv')
df_store = df_store.fillna(-1)

df_train = df_train[df_train.Sales > 0]
df_train = df_train[df_train.Customers > 0]

df_train_store = df_train.join(df_store.set_index('Store'), on='Store')
df_train_store = substract_cols(df_train_store)
df_train_store.drop(['Open','PromoInterval','Year','Month','WeekOfYear'], axis=1, inplace=True)

df_test_store = df_test.join(df_store.set_index('Store'), on='Store')
df_test_store = substract_cols(df_test_store)
df_test_store.drop(['Open','PromoInterval','Year','Month','WeekOfYear'], axis=1, inplace=True)



df = df_train_store
labels = df['Sales']
df.drop(['Sales'], axis=1, inplace=True)
df_train_store_with_out_sales = df
frames = [df_train_store_with_out_sales, df_test_store]
super_df = pd.concat(frames)

mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
super_df.StateHoliday.replace(mappings, inplace=True)
super_df.StoreType.replace(mappings, inplace=True)
super_df.Assortment.replace(mappings, inplace=True)

# scaler = MinMaxScaler() 
# scaled_values = scaler.fit_transform(super_df) 
# super_df.iloc[:,2] = scaled_values[:,2]
# super_df.iloc[:,11:14] = scaled_values[:,11:14]

super_df = pd.get_dummies(super_df, columns=['Store', 
                                                         'DayOfWeek',
                                                         'StateHoliday',
                                                         'Day',
                                                         'StoreType',
                                                         'Assortment'])

start_ind = df_train_store.shape[0]
df_train_store = super_df.iloc[:start_ind,:]
df_test_store = super_df.iloc[start_ind:,:]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
super_df.head()

,Customers,Promo,SchoolHoliday,CompetitionDistance,Promo2,CompetitionOpen,PromoOpen,Store_1,Store_2,Store_3,...,Day_29,Day_30,Day_31,StoreType_1,StoreType_2,StoreType_3,StoreType_4,Assortment_1,Assortment_2,Assortment_3
0,616,1,0,1270.0,0,76.0,0.00,1,0,0,...,0,1,0,0,0,1,0,1,0,0
1,624,1,0,570.0,1,86.0,58.00,0,1,0,...,0,1,0,1,0,0,0,1,0,0
2,678,1,0,14130.0,1,97.0,45.75,0,0,1,...,0,1,0,1,0,0,0,1,0,0
3,1632,1,0,620.0,0,64.0,0.00,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,617,1,0,29910.0,0,0.0,0.00,0,0,0,...,0,1,0,1,0,0,0,1,0,0


In [5]:
df_test_store.head()

,Customers,Promo,SchoolHoliday,CompetitionDistance,Promo2,CompetitionOpen,PromoOpen,Store_1,Store_2,Store_3,...,Day_29,Day_30,Day_31,StoreType_1,StoreType_2,StoreType_3,StoreType_4,Assortment_1,Assortment_2,Assortment_3
0,555,1,1,1270.0,0,82.0,0.00,1,0,0,...,0,0,1,0,0,1,0,1,0,0
1,625,1,1,570.0,1,92.0,64.50,0,1,0,...,0,0,1,1,0,0,0,1,0,0
2,821,1,1,14130.0,1,103.0,52.25,0,0,1,...,0,0,1,1,0,0,0,1,0,0
3,1498,1,1,620.0,0,70.0,0.00,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,559,1,1,29910.0,0,3.0,0.00,0,0,0,...,0,0,1,1,0,0,0,1,0,0


In [11]:
X = df_train_store.values
y = labels.values

X_test = df_test_store.values

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.85, random_state=1234)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
out_dims = [1100]
bss = [2000]
num_epochs = 100
dropout = 0.2
verb = 1
for out_dim in out_dims:
    for bs in bss:
        model = Sequential()
        model.add(Dense(output_dim=out_dim, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dropout(dropout))
#         model.add(Dense(output_dim=20,kernel_initializer='normal'))
        model.add(Dense(output_dim=1, kernel_initializer='normal'))
        model.compile(loss='mean_squared_error', optimizer='adam')
        
        start = time.time()
        model.fit(X, y, 
        verbose=verb, 
        nb_epoch=num_epochs, 
        batch_size=bs,
        validation_data=(X_validation, y_validation))
        end = time.time()
        
#         pred = model.predict(X_validation)
#         accuracy = rmspe(y_validation, pred)
        
        

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=1170, kernel_initializer="normal", activation="relu", units=1100)`
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="normal", units=1)`
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 54688 samples, validate on 8204 samples
Epoch 1/100
54688/54688 [==============================] - 10s - loss: 52188875.1738 - val_loss: 40668787.0502
Epoch 2/100
54688/54688 [==============================] - 11s - loss: 33217952.6466 - val_loss: 23778441.4130
Epoch 3/100
54688/54688 [==============================] - 10s - loss: 16436600.3508 - val_loss: 9113405.2277
Epoch 4/100
54688/54688 [==============================] - 10s - loss: 5959925.9579 - val_loss: 3822774.9032
Epoch 5/100
54688/54688 [==============================] - 10s - loss: 3803725.8726 - val_loss: 3417710.0334
Epoch 6/100
54688/54688 [==============================] - 10s - loss: 3652212.1728 - val_loss: 3321331.3784
Epoch 7/100
54688/54688 [==============================] - 10s - loss: 3523461.1202 - val_loss: 3196833.6714
Epoch 8/100
54688/54688 [==============================] - 10s - loss: 3407781.4798 - val_loss: 3069442.7590
Epoch 9/100
54688/54688 [==============================] - 9s - loss: 3271

54688/54688 [==============================] - 9s - loss: 2316124.3265 - val_loss: 2144031.2159
Epoch 76/100
54688/54688 [==============================] - 9s - loss: 2295813.5430 - val_loss: 2119007.1921
Epoch 77/100
54688/54688 [==============================] - 9s - loss: 2292291.3433 - val_loss: 2109767.2984
Epoch 78/100
54688/54688 [==============================] - 9s - loss: 2286933.3132 - val_loss: 2079867.1253
Epoch 79/100
54688/54688 [==============================] - 10s - loss: 2268932.1372 - val_loss: 2068798.4534
Epoch 80/100
54688/54688 [==============================] - 9s - loss: 2255380.9768 - val_loss: 2053655.2448
Epoch 81/100
54688/54688 [==============================] - 9s - loss: 2245229.8950 - val_loss: 2049209.2613
Epoch 82/100
54688/54688 [==============================] - 9s - loss: 2227699.2322 - val_loss: 2042098.5858
Epoch 83/100
54688/54688 [==============================] - 10s - loss: 2218323.1966 - val_loss: 2020599.0099
Epoch 84/100
54688/54688 [====

In [14]:
nic = pd.read_csv('nic_submission.csv')
nic_pred = nic.Sales

pred_test = model.predict(X_test)

for i in range (X_test.shape[0]) :
    if df_test_store['Customers'].iloc[i] == 0 :
        pred_test[i] = 0
        
accuracy = rmspe(nic_pred, pred_test)

print('---------//---------')
print('output     ',out_dim)
print('batch size ',bs)
print('epochs ',num_epochs)
# print('time taken ',end-start)
print("RMSPE is ",accuracy)
print()

---------//---------
output      1100
batch size  2000
epochs  100
RMSPE is  0.1136839385038404



In [15]:
pred_test = model.predict(X_test)
pred_test = np.reshape(pred_test, pred_test.shape[0])
submission = pd.DataFrame()
submission['Sales'] = pred_test
cols = ['Id','Sales']
submission['Id'] = submission.index + 1
submission = submission[cols]
submission.to_csv('submission.csv', index=False)